# Iteration implementieren

## Agenda

1. Rückblick: Iteration
2. Einzelheiten: *iterables*, *iterators*, `iter`, und `next`
3. Implementierung von Iteratoren mit Klassen
4. Implementierung von Iteratoren mit *Generatoren* und `yield`

# Was ist Iteration?

Unter *Iteration* versteht man einfach den Prozess des Zugriffs - einer nach dem anderen - auf die in einem Container gespeicherten Elemente. Die Reihenfolge der Elemente und ob die Iteration umfassend ist oder nicht, hängt vom Container ab.

In Python wird die Iteration in der Regel mit der Schleife "for" durchgeführt.

In [ ]:
# e.g., iterating over a list
l = [2**x for x in range(10)]
for n in l:
    print(n)

In [ ]:
# e.g., iterating over the key-value pairs in a dictionary
d = {x:2**x for x in range(10)}
for k,v in d.items():
    print(k, '=>', v)

## Review: *iterables*, *iterators*, `iter`, and `next`

Wir können über alles iterieren, was *iterable* ist. Intuitiv ist etwas, das als Quelle von Elementen in einer "for"-Schleife verwendet werden kann, iterierbar.

Aber wie funktioniert eine "for"-Schleife wirklich? (Zeit für einen Rückblick!)
Suchen Sie in der Python-Dokumentation nach den Objekten [iter()](https://docs.python.org/3/library/functions.html#iter) und [iterator](https://docs.python.org/3/glossary.html#term-iterator).

In [ ]:
a = 'Hi'
itr = iter(a)
type(itr)

In [ ]:
next(itr)

In [ ]:
next(itr)

In [ ]:
next(itr)

Was wir also wirklich machen:

In [ ]:
l = [2**x for x in range(10)]

itr = iter(l)
while True:
    try:
        n = next(itr)
        print(n)
    except StopIteration:
        break

# Iterator Klassen implementieren

In [ ]:
class MyIterator:
    def __init__(self, max):
        self.max = max
        self.curr = 0
        
    # the following methods are required for iterator objects
    
    def __next__(self):
        if self.curr < self.max:
            self.curr += 1
            return self.curr - 1
        else:
            raise StopIteration("maximum iteration level reached")
    
    def __iter__(self):
        return self

In [ ]:
it = MyIterator(5)

In [ ]:
next(it)

In [ ]:
it = MyIterator(10)
while True:
    try:
        print(next(it))
    except StopIteration:
        break

In [ ]:
it = MyIterator(10)
for i in it:
    print(i)

Ein Iterator ist ein *Einmalgebrauchsobjekt*! D.h., wenn wir ihn einmal benutzt haben, um über Elemente zu iterieren, können wir die Iteration normalerweise nicht zurücksetzen oder "zurückspulen". Iterable-Objekte, die wiederholt durchlaufen werden können, geben bei jedem Durchlauf neue Iteratoren zurück.

In [ ]:
l = ['a', 'b', 'c', 'd', 'e']
for _ in range(3):
    for x in l:
        print(x, end=' ')

In [ ]:
l = ['a', 'b', 'c', 'd', 'e']
for _ in range(3):
    it = iter(l) # we obtain and "use up" an iterator each loop!
    while True:
        try:
            x = next(it)
            print(x, end=' ')
        except StopIteration:
            break

Für eine beliebige Container-Klasse benötigen wir eine `__iter__()` Methode, welche ein Iterator-Objekt zurückliefert.

In [ ]:
class SnackBox:
    def __init__(self, snacks):
        self.snacks = snacks
        self.index = 0

    def __iter__(self):
        class SnackBoxIterator:
            def __init__(self, data):
                self.data = data
                self.index = 0        
        
            def __next__(self):
                if self.index < len(self.data):
                    snack = self.data[self.index]
                    self.index += 1
                    return snack
                else:
                    raise StopIteration

            def __iter__(self):
                return self

        return SnackBoxIterator(self.snacks)
                

In [ ]:
# Beispiel für die Verwendung der SnackBox
snack_box = SnackBox(["Chips", "Schokolade", "Gummibärchen", "Nüsse", "Kekse"])

for snack in snack_box:
    print(snack)

In [ ]:
it = iter(snack_box)
type(it)

# Iteratoren mit Generatoren implementieren

In der Einführung hatten wir *Generatoren* besprochen. 

In [ ]:
# Liste vs Generator
l = [2*x for x in range(10)]
g = (2*x for x in range(10))

In [ ]:
next(g)

# Generator-Funktionen bauen mit `yield

In [ ]:
def foo():
    yield

In [ ]:
foo()

In [ ]:
type(foo())

In [ ]:
def foo():
    print('hello!')
    yield
    print('goodbye!')

In [ ]:
foo()

In [ ]:
g = foo()

In [ ]:
next(g)

In [ ]:
next(g)

In [ ]:
def foo():
    yield 1
    yield 2
    yield 3

In [ ]:
g = foo()

In [ ]:
next(g)

In [ ]:
for x in foo():
    print(x)

In [ ]:
def countdown(n):
    for x in range(n, 0, -1):
        yield x
    yield 'Boom!'

In [ ]:
for x in countdown(5):
    print(x)

In [ ]:
list(countdown(10))

Eine *Generatorfunktion* ist eine Funktion, die eine oder mehrere `yield`-Anweisungen enthält. Wenn eine Generatorfunktion aufgerufen wird, gibt sie ein Generatorobjekt zurück, das es uns ermöglicht, die Funktion mithilfe der Iterations-API schrittweise auszuführen. Jeder Aufruf von `next` auf dem Generator führt die Funktion bis zur nächsten `yield`-Anweisung aus; wenn die Funktion beendet ist, löst der Generator eine `StopIteration`-Ausnahme aus (genau wie ein Iterator).

Damit eignen sich Generatoren hervorragend zum iterieren:

In [ ]:
class SnackBox:
    def __init__(self, snacks):
        self.snacks = snacks
        self.index = 0

    def __iter__(self):
        for i in range(len(self.snacks)):
            yield self.snacks[i]

In [ ]:
# Beispiel für die Verwendung der SnackBox
snack_box = SnackBox(["Chips", "Schokolade", "Gummibärchen", "Nüsse", "Kekse"])

for snack in snack_box:
    print(snack)

#### Aufgabe 1: Einfacher Iterator

Erstellen Sie eine Klasse `NumberIterator`, die eine Liste von Zahlen von 1 bis 10 enthält. Implementieren Sie die Methoden `__iter__()` und `__next__()`, sodass man über die Zahlen iterieren kann.


In [ ]:
class NumberIterator:
    def __init__(self):
        self.numbers = list(range(1, 11))
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.index < len(self.numbers):
            number = self.numbers[self.index]
            self.index += 1
            return number
        else:
            raise StopIteration

In [ ]:
number_iterator = NumberIterator()
for number in number_iterator:
    print(number)

#### Aufgabe 2: Generator mit `yield`

Erstellen Sie eine Generatorfunktion `even_numbers(n)`, die die ersten `n` geraden Zahlen zurückgibt. Testen Sie die Funktion, indem Sie die ersten 10 geraden Zahlen ausgeben.


In [ ]:
def even_numbers(n):
    for i in range(n):
        yield 2 * i

In [ ]:
for number in even_numbers(10):
    print(number)

#### Aufgabe 3: Parkplatzverwaltung in Python

Implementiere ein **Parkplatzverwaltungssystem**, das Autos **parken, entfernen und iterieren** kann.  
Nutze **`yield` für die Iteration** der geparkten Autos.

Vervollständige hierfür die Methoden in den vorgegebenen Klassen: 

In [5]:
class Car:
    """
    Represents a car with model, manufacturing year, and license plate.
    """
    def __init__(self, model, year, license_plate):
        self.model = model
        self.year = year
        self.license_plate = license_plate

    def __str__(self):
        return f"{self.year} {self.model} with license plate {self.license_plate}"

In [6]:
class ParkingLot:
    """
    Represents a parking lot with a fixed capacity for parking cars.
    """
    def __init__(self, capacity):
        """
        Initializes a ParkingLot instance.
        Saves the capacity and initializes an empty list of cars.

        Args:
            capacity (int): The maximum number of cars the parking lot can hold.
        """
        self.capacity = capacity
        self.cars = []
        

    def park_car(self, car):
        """
        Attempts to park a car in the parking lot.

        Args:
            car (Car): The car to be parked.

        Prints:
            A message indicating whether the car was parked successfully or if the lot is full.
        """
        if len(self.cars) < self.capacity:
            self.cars.append(car)
            print(f"Parked {car}")
        else:
            print("Parking lot is full")

    def leave_car(self, car):
        """
        Removes a car from the parking lot if it is present.

        Args:
            car (Car): The car to be removed from the lot.

        Prints:
            A message indicating whether the car has left the lot or if it was not found.
        """        
        if car in self.cars:
            self.cars.remove(car)
            print(f"🚗 {car} left the parking lot")
        else:
            print(f"⚠️ Car with license plate {car.license_plate} is not in the parking lot")
        
    def __iter__(self):
        """
        Allows iteration over the parked cars.

        Yields:
            Car: The next car in the parking lot.
        """        
        for car in self.cars:
            yield car


In [7]:
parking_lot = ParkingLot(capacity=2)

car1 = Car("Tesla Model 3", 2022, "ABC-123")
car2 = Car("BMW X5", 2020, "XYZ-789")
car3 = Car("Audi A4", 2019, "DEF-456")

parking_lot.park_car(car1)
parking_lot.park_car(car2)
parking_lot.park_car(car3)  # Sollte "Parkhaus voll" ausgeben

print("Geparkte Autos:")
for car in parking_lot:
    print(car)

Parked 2022 Tesla Model 3 with license plate ABC-123
Parked 2020 BMW X5 with license plate XYZ-789
Parking lot is full
Geparkte Autos:
2022 Tesla Model 3 with license plate ABC-123
2020 BMW X5 with license plate XYZ-789
